In [ ]:
!pip install langchain openai chromadb lark langchain-community sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.6/987.6 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.5/366.5 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
import json
import warnings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from google.colab import userdata
import google.generativeai as genai
from langchain.llms import GooglePalm

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
api=userdata.get('gGOOGLE_API_KEY')

In [ ]:
with open("/content/agentic_ai.json", "r") as f:
    tutorial_data = json.load(f)

In [ ]:
#keep the hugging face token in the secrets
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
chunks = []

# Add the overall goal of the tutorial
chunks.append(Document(
    page_content=tutorial_data["goal"],
    metadata={"id":"goal","title":"goal of the tutorial","tags":"goal,learn","type": "goal"}
))
# Chunk each concept
for concept in tutorial_data["concepts"]:
    if concept['id'] in ["walkthrough","callbacks"]:
       content =  f"{concept['title']}\n"
       for step,code in zip(concept['content'],concept['code']):
           content += f"- {step}\ncode example:{code}\n"
    else:
      content = f"{concept['title']}\n{concept['content']}"

      if 'code' in concept:
          content += f"\nCode Example:\n{concept['code']}"

      if 'state_table' in concept:
          content += "\nState Table:\n"
          for state_info in concept['state_table']:
              content += f"- **{state_info['state']}**: {state_info['description']}\n"

      if 'formats' in concept:
          content += "\nFormats:\n"
          for example in concept['formats']:
              content += f"- **Format:** {example['format']}, **Extension:** {example['extension']}\n"

    chunks.append(Document(
        page_content=content,
        metadata={
            "id": concept['id'],
            "title": concept['title'],
            "tags": ", ".join(concept.get('tags', [])),  # Convert tags list to string
            "related_concepts": ", ".join(concept.get('related_concepts', [])),  # Convert to string
            "type": "concept"
        }
    ))

# Chunk codes
for code_example in tutorial_data["codes"]:
    chunks.append(Document(
        page_content=f"{code_example['title']}\n{code_example['content']}\n{code_example.get('explanation', [])}",
        metadata={
            "id": code_example['id'],
            "title": code_example['title'],
            "tags": ", ".join(code_example.get('tags', [])),  # Convert tags list to string
            "related_concepts": ", ".join(code_example.get('related_concepts', [])),  # Convert to string
            "type": "code_example"
        }
    ))

# Chunk exercises
for exercise in tutorial_data["exercises"]:
    hints_str = "\n".join(exercise.get('hints', []))  # Join hints into a string
    chunks.append(Document(
        page_content=f"{exercise['title']}\n{exercise['content']}\nHints:\n{hints_str}",
        metadata={
            "id": exercise['id'],
            "title": exercise['title'],
            "related_concepts": ", ".join(exercise.get('related_concepts', [])),  # Convert to string
            "tags": ", ".join(exercise.get('tags', [])),  # Convert tags list to string
            "type": "exercise"
        }
    ))
#chunk conclusion
for conclusion in tutorial_data["conclusion"]:
    conclusion_content = f"{conclusion['title']}\n{conclusion['content']}"
    chunks.append(Document(
        page_content=conclusion_content,
        metadata={
            "id": "conclusion",
            "title": "conclusion for the tutorial",
            "tags": "conclusion",

            "type": "conclusion"
        }
    ))
# 3.Ceate Vector Store
vectorstore = Chroma.from_documents(chunks, embeddings)

In [ ]:
#Info for each metadata field
metadata_field_info = [
    AttributeInfo(
        name="id",
        description="Unique identifier for each document section.",
        type="string",
    ),
    AttributeInfo(
        name="title",
        description="It is the title of that document which indicates main context in the document",
        type="string",
    ),
    AttributeInfo(
        name="tags",
        description="Comma-separated list of tags associated with the document section(each tag indicates that data related to that element are present in the document)",
        type="string",
    ),
    AttributeInfo(
    name="related_concepts",
    description="A comma-separated list of IDs representing other concepts that are directly related to this one. This allows for creating a network of interconnected concepts within the data.",
    type="string",
    ),
    AttributeInfo(
        name="type",
        description="Categorization of the document section's content (e.g., 'concept:it contains the detail explanation of particular concept', 'code: it contains the code'",
        type="string"
    ),
]

In [ ]:
llm = GooglePalm(
    model_name="models/gemini-1.5-pro",
    temperature=0.7,
    max_output_tokens=1024,
    top_p=0.95,
    top_k=40,
    google_api_key=api#replace with your api key
)

In [ ]:
document_content_description = "This document provides a tutorial on building dynamic pipelines using GStreamer."
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    filter_func=filter_complex_metadata
)

In [ ]:
template = """Use the following pieces of context to answer the question at the end.
Try your best to formulate a comprehensive answer using the provided information and the code in this tutorial is c programming and use content only from the below context.only give answer for the above question if the answer is in the below context dont give extra content or irrelavent content.
If you don't know the answer, just say that you don't know, don't try to make up an answer

{context}

Question: {question}
Answer:"""


QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"], template=template
)


chain=RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
while True:
    question = input("User: ")
    if question.lower() == "exit":
        break
    output = chain(question)
    answer = output['result']
    docs = output['source_documents']
    print("Chatbot: ",answer)

User: explain about demuxers
Chatbot:  Demuxers are elements responsible for opening container files that store audio and video together (multiplexed, or muxed). They separate multiple streams within the container (e.g., one video and two audio tracks) and expose them through different output ports. This allows for creating different branches in the pipeline to handle each type of data separately. 

User: Explain about pads
Chatbot:  The ports through which GStreamer elements communicate with each other are called pads (GstPad). Elements have sink pads for data entering an element and source pads for data exiting an element. Source elements only contain source pads, sink elements only contain sink pads, and filter elements contain both. 

User: what are the different types of demuxer formats
Chatbot:  - **Format:** Matroska, **Extension:** MKV
- **Format:** QuickTime, **Extension:** MOV
- **Format:** Ogg, **Extension:** ogg
- **Format:** Advanced Systems Format, **Extension:** ASF, WMV

Chatbot:  ```c
typedef struct _CustomData {
  GstElement *pipeline;
  GstElement *source;
  GstElement *convert;
  GstElement *resample;
  GstElement *sink;
} CustomData;
``` 
This code defines a structure named `CustomData` to hold pointers to various GstElements used in the pipeline. 

User: what is Gsignal
Chatbot:  GSignals are a crucial part of GStreamer. They allow you to be notified (by means of a callback function) when something interesting has happened in the pipeline. Signals are identified by a name, and each GObject (the base object type in GStreamer) has its own set of signals. 

User: What are the different types gstreamer states


Chatbot:  There are four states in GStreamer: **NULL**, **READY**, **PAUSED**, and **PLAYING**. 

User: what is the use of gst_structure_get_name()
Chatbot:  The code snippet you provided uses `gst_structure_get_name()` to get the name of the GstStructure representing the media type of a GStreamer pad. 

**Specifically, it's used to determine if the newly added pad from the 'uridecodebin' element is an audio stream.**

Here's the relevant part of the code:

```c
  /* Check the new pad's type */
  new_pad_caps = gst_pad_get_current_caps (new_pad);
  new_pad_struct = gst_caps_get_structure (new_pad_caps, 0);
  new_pad_type = gst_structure_get_name (new_pad_struct);
  if (!g_str_has_prefix (new_pad_type, "audio/x-raw")) {
    g_print ("It has type '%s' which is not raw audio. Ignoring.\n", new_pad_type);
    goto exit;
  }
```

**Explanation:**

1. **`gst_pad_get_current_caps(new_pad)`:** This function retrieves the capabilities of the newly added pad (`new_pad`). Capabilities describe wh